In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))

# Read prepped cohorts data

- Read in prepared data and output

In [ ]:
base <- fread(paste0(SHARE_DIR, "2_run_marginal_output.csv")) 

In [ ]:
cohorts <- fread(paste0(SHARE_DIR, "top_mechanisms.csv"))

In [ ]:
treatment_mechanism_map <- 
fread(paste0(SHARE_DIR, "treatment_mechanism_map.csv")) %>% 
 mu(derived_treatmentName = gsub( "##","/", derived_treatmentName),
    derived_treatmentMechanism = gsub( "##","/", derived_treatmentMechanism))

# Let's go
- Add fields

In [ ]:
low_responder_threshold <- .1
pval_threshold <- .1

In [ ]:
orderer <- 
c('Anti-AR',
  'Anti-CTLA-4 / Anti-PD-1', 
  'Anti-PD-1', 
  'Immunotherapy', 
  'Alkylating / Anthracycline',
  'Chemotherapy',
  'Multikinase inhibitor',
  'Targeted therapy',
  "Non-significant/Other")

In [ ]:
lets_go <- 
base %>% 
 mu(p_fdr_fisher = p.adjust(fisher_pval, method = "fdr"), 
    p_fdr_fisher_by = p.adjust(fisher_pval, method = "BY"),
    p_fdr_surv = p.adjust(surv_pval, method = "fdr"),
    or = ifelse(or == "Inf", exp(5), or), 
    surv_high = surv_est + 1.96*surv_se, 
    surv_low = surv_est - 1.96*surv_se,
    prob_response = e_r/events,
    low_responder = (prob_response <= low_responder_threshold),
    sig_fisher = (p_fdr_fisher <= pval_threshold),
    sig_pfs = (p_fdr_surv <= pval_threshold),
    highlight = ((low_responder) & (sig_fisher)), 
    super_highlight = ((highlight) & (sig_pfs)),
    Odds = ifelse(direction == "Response", "Better", "Worse"),
    pan = grepl("Pan-Cancer", cohortGo)) %>% 
  rw() %>% mu( derived_treatmentName = str_split_fixed( cohortGo	, " / ", n = 2)[2]) %>% ug() %>% 
  lj(cohorts %>% se(cohortGo, group), by = "cohortGo") %>% 
  lj( treatment_mechanism_map , by = "derived_treatmentName") %>% 
  mu( derived_treatmentMechanism = ifelse(is.na(derived_treatmentMechanism), derived_treatmentName, derived_treatmentMechanism),
      treatment = derived_treatmentName, 
      mechanism = gsub(" ## ", "/", derived_treatmentMechanism),
      example = (cohortGo != "Pan-Cancer" & p_fdr_fisher <= pval_threshold/10 & p_fdr_surv < pval_threshold/10 & Odds == "Worse" & mechanism != "Chemotherapy"), 
      Treatment = factor(ifelse(highlight, mechanism, "Non-significant/Other"), levels = orderer)  ) %>% 
  se(-direction)

# Nice Labels

- Need to process names for downstream plots

In [ ]:
highlights <- lets_go %>% fi(or < 1, (p_fdr_fisher < .2) | (grepl("driver", feature)), low_responder) %>% pu(feature) %>% unique()

In [ ]:
#highlights

In [ ]:
replacements <- 
c("_" = " ", 
  "rna geneset " = "RNA", 
  "gene set" = "",
  "HALLMARK" = "", 
  "KEGG" = "",
  "gt0" = "",
  "gt75" = "Very High",
  "gt50" = "High",
  "gt25" = "Moderate/High",
  "lt75" = "Low/Moderate",
  "lt50" = "Low",
  "lt25" = "Very Low",
  "hotspot KRAS position 25398284" = "KRAS G12D hotspot",
  "purity tmbPerMb lt6" = "TMB per MB < 6",
  "neo ct" = "RNA Neoantigens",
  "TGF BETA SIGNALING PATHWAY" = "TGFB",
  "SIGNALING " = "",
  " AND " = "/")

# Loop over names and replace
s1 <- highlights
for (pattern in names(replacements)) {
  s1 <- gsub(pattern, replacements[pattern], s1)
}

In [ ]:
updates <- 
c("Kras" = "KRAS", "Rna" = "RNA", "Tim" = "TIM", "Tgfb" = "TGFB", "G12d" = "G12D", "Erbb" = "ERBB", "Wnt" = "WNT",
  "Nrf2" = "NRF2",  "Ddr" = "DDR")

In [ ]:
ready <- str_to_title(s1)
for (pattern in names(updates)) {
  ready <- gsub(pattern, updates[pattern], ready)
}

In [ ]:
name_map <- as.list(trimws(ready))
names(name_map) <- highlights

In [ ]:
next_step <- 
lets_go %>% 
 rw() %>% 
 mu(clean_name = ifelse(feature %in% names(name_map), name_map[[feature]], feature)) %>% 
 ug()

In [ ]:
#next_step %>% fi(grepl("driver", feature)) %>% se(clean_name, feature)

# Send processed data

In [ ]:
saveRDS(next_step, paste0(SHARE_DIR, "3_ready_marginal.Rds"))